<a href="https://colab.research.google.com/github/mmovahed/Spectral_Methods/blob/main/ODE/Subdomain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Subdomain weighted residual method

Please consider initial value problem:

$y'=y$

$y(0)=1$

with exact solution:

$y=e^x$

In [3]:
import numpy as np
from scipy.special import chebyt
from scipy.integrate import quad
from scipy.linalg import solve

# Define the ODE as a function
def ode(x, y):
    return y  # Example: dy/dx = y

def exact(x):
    return np.exp(x)

# Initial condition
x0 = 0
y0 = 1

# Define the basis functions (Chebyshev polynomials)
def T(n, x):
    return chebyt(n)(x)

# Number of basis functions (order of the polynomial)
N = 10

# Rescale x to [-1, 1] for Chebyshev polynomials
def rescale(x, a, b):
    return (2 * x - (b + a)) / (b - a)

# Define the number of subdomains
num_subdomains = 10
subdomain_length = (x0 + 1 - x0) / num_subdomains

# Compute the matrix A and vector b for the subdomain weighted residual method
A = np.zeros((N + 1, N + 1))
b = np.zeros(N + 1)

for k in range(num_subdomains):
    a_k = x0 + k * subdomain_length
    b_k = a_k + subdomain_length

    for i in range(N + 1):
        for j in range(N + 1):
            integrand_A = lambda x: T(i, rescale(x, a_k, b_k)) * T(j, rescale(x, a_k, b_k))
            A[i, j] += quad(integrand_A, a_k, b_k)[0]

        integrand_b = lambda x: T(i, rescale(x, a_k, b_k)) * ode(x, exact(x))
        b[i] += quad(integrand_b, a_k, b_k)[0]

# Solve the weighted residual problem
coeffs = solve(A, b)

# Define the approximate solution
def approx_solution(x):
    x_rescaled = rescale(x, x0, x0 + 1)
    y = sum(coeffs[i] * T(i, x_rescaled) for i in range(N + 1))
    return y

# Test the solution at a new point
x_test = 0.5
y_test = approx_solution(x_test)
print(f"The approximate solution at x = {x_test} is y = {y_test}")
y_exact = exact(x_test)
print(f"The exact solution at x = {x_test} is y = {y_exact}")
print(f"MAE at x = {x_test} is {np.abs(y_exact - y_test)}")

The approximate solution at x = 0.5 is y = 1.7175660864611293
The exact solution at x = 0.5 is y = 1.6487212707001282
MAE at x = 0.5 is 0.0688448157610011
